<a href="https://colab.research.google.com/github/GdRottoli/RequirementPriorization/blob/main/Requirement_Priorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-criteria Requirement Priorization based on the opinion of many experts using fuzzy linguistic labels

Below, there is the implementation of a multi-criteria requirement priorization (RP) method, using as imput the opinion of many experts on many dimensions.

Each dimension and each expert can be ponderated differently.

The experts' opinions as well as the dimensions and experts ponderations are expresed using a linguistic scale specified by the decision-maker. 

The experts' opinions are aggregated considering the frequency of the opinions provided for a requirement on a specific dimension. This is performed using the Majority Guided Induced OWA (MLIOWA).

----

## How to use

In order to use this notebook, first see the example xls file used as input. That file contains for each expert a sheet with their opinions. **It is important to use the same exact format.**

Then, in the Configuration area, the user need to specify the following values:

*   `S:` Linguistic labels, ordered from lowest to highest value.
*   `experts_importance`: importance of each expert, it is important specify the values in the exact order as in the input file. 
*   `dimensions_weights`: for each dimension, its important can be specified using linguistic labels from S or using numbers. All the dimensions MUST be listed. 
*   `data_file_path`: the path to the input file. 

Then, the whole notebook can be executed in one go (`Ctrl+F9`) or cell by cell. 

The last cell shows the partial order of the requirements. 

----

For suggestions or comments please send an e-mail to:
*   gd.rottoli@gmail.com
*   carlos.casanova16@gmail.com 

Finally, we apologise for our English!



# Configuration

In [368]:
S = ('Very Low', 'Low', 'Middle', 'High', 'Very High')

In [369]:
experts_importance = ['Middle', 'Very High', 'Middle', 'Middle', 'Middle']

In [370]:
# Same column names as in the input file
# The values should be float or belong to S. DO NOT mix float and labels.
dimensions_weights = { 
    'Complexity': 'Very High',
    'Reusability': 'Middle',
    'Importance':  'Middle'
}

In [371]:
# See attached file for example
data_file_path = 'Data_ReqPri_1E.xls' 

# Excecution

In [372]:
#@title Imports {display-mode: "form"}

import numpy as np
import pandas as pd
import itertools

In [373]:
#@title Data Loading {display-mode: "form"}
xls = pd.ExcelFile(data_file_path)
df = pd.concat(pd.read_excel(xls, sheet_name=None))
df = df.droplevel(1).reset_index()

## Functions 

In [374]:
#@title Basic fuzzy functions {display-mode: "form"}

labelValue = lambda x: S.index(x)

labelMax = lambda x,y: S[max(labelValue(x), labelValue(y))]
labelMin = lambda x,y: S[min(labelValue(x), labelValue(y))]
neg = lambda x: S[len(S) - labelValue(x)  - 1]
# Implication
imp = lambda x, y : labelMax(neg(x), y)

In [375]:
#@title Fuzzy Quantifier {display-mode: "form"}
def most(x, li = 0.3, ls = 0.8, exp=1.0):
  if x < li: 
    return 0
  elif x >= ls :
    return 1
  else:
    return  ((x - li) / (ls - li))**exp

In [376]:
#@title Support function {display-mode: "form"}
def support(I, alpha=1):
  sup = []
  for i in I:
    sup.append(sum([1 for j in I if abs(labelValue(i) - labelValue(j)) <= alpha]))
  return sup

In [377]:
#@title Majority Guided IOWA {display-mode: "form"}
def MLIOWAPond(I, P):
  '''
    MLIOWAPond: S x S --> S
    I: importance degree
    P: expert's opinion
  '''
  # Induced Order
  U = (np.array(support(I, alpha=1)) + np.array([labelValue(i) for i in I])) / 2
  Us, Ps = zip(*sorted(zip(U, P), key = lambda t: t[0]))
  # Weights
  # Error if most = 0 --> division by 0
  # Use implication
  W_aux = list(map(most,np.array(Us)/len(Us)))
  W = np.array(W_aux) / sum(W_aux)
  IndPs = list(map(labelValue, Ps))
  k = sum(np.array(W) * np.array(IndPs))
  return S[round(k)]

## Aggregation

In [378]:
# Apply expert importance to each
agg_func = lambda x : MLIOWAPond(experts_importance, x)
dfagg = df.groupby('Id').aggregate({'index':sum,
                            'Complexity':agg_func,
                            'Reusability':agg_func,
                            'Importance':agg_func})
dfagg = dfagg.drop('index', 1)

# Ponderate dimensions if linguistic weights
if type(list(dimensions_weights.values())[0]) is str:
  for column in dfagg.columns:
    dfagg[column] = list(map(imp, [dimensions_weights[column]]*len(dfagg[column]), dfagg[column]))
dfagg

,Complexity,Reusability,Importance
Id,,,
1,Middle,Middle,High
2,Middle,Very High,Very High
3,Very High,Middle,High
4,Very Low,Middle,Middle
5,Middle,High,High
6,Very High,Very High,High
7,Low,Middle,Middle
8,Very Low,Middle,Middle
9,Very Low,Middle,Middle


##Priorization

In [379]:
comparison_mu = lambda x: 1 if labelValue(x[0]) >= labelValue(x[1]) else (0.5 if labelValue(x[0]) == (labelValue(x[1]) - 1) else 0)

In [380]:
indexes = dfagg.index.values
global_comparison_matrix = []
order = []

for column in dfagg.columns:
  # Cross product
  cross_product = list(itertools.product(dfagg[column],dfagg[column]))
  # Comparison matrix
  comparison_matrix = np.array(list(map(comparison_mu, cross_product)))
  # Ponderate according to dimension
  if type(list(dimensions_weights.values())[0]) in [float, int]:
    comparison_matrix = comparison_matrix ** dimensions_weights[column]
  # Global comparison matrix (T-norm min)
  if len(global_comparison_matrix) == 0:
    global_comparison_matrix = comparison_matrix
  else:
    global_comparison_matrix = np.minimum(global_comparison_matrix, comparison_matrix)

# Reshape: turn vector into matrix
dim = int(len(indexes))
global_comparison_matrix = global_comparison_matrix.reshape((dim,dim))

# Extrict relation
strict_relation = global_comparison_matrix - global_comparison_matrix.T
strict_relation[strict_relation < 0] = 0 

# Non dominance vector
while len(indexes) > 0:
  non_dominance_vector = 1 - np.amax(strict_relation, 0)
  non_dominated = np.where(non_dominance_vector == max(non_dominance_vector))
  # get non dominated requirements
  order.append(list(indexes[non_dominated]))
  # remove non_dominated from matrix and indexes
  indexes = np.delete(indexes, non_dominated)
  strict_relation = np.delete( np.delete(strict_relation, non_dominated, 0), non_dominated, 1)

# Output

In [381]:
print("-----------------------------")
print(" Requirement's Priorization ")
print("-----------------------------")
for i, l in enumerate(order):
  print("Position ", i+1, ": Requirements: ", l )

-----------------------------
 Requirement's Priorization 
-----------------------------
Position  1 : Requirements:  [6]
Position  2 : Requirements:  [2, 3]
Position  3 : Requirements:  [5, 10]
Position  4 : Requirements:  [1]
Position  5 : Requirements:  [7]
Position  6 : Requirements:  [4, 8, 9]
